In [2]:
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv,dotenv_values
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [2]:
FILE_PATH = "C:/Ambarish/NCERT/CHAP04-BIOLOGY-CLASS11.pdf"

In [3]:
# Read the PDF file and return the text
def get_pdf_data(file_path, num_pages = 1):
    reader = PdfReader(file_path)
    full_doc_text = ""
    pages = reader.pages
    num_pages = len(pages) 
    
    try:
        for page in range(num_pages):
            current_page = reader.pages[page]
            text = current_page.extract_text()
            full_doc_text += text
    except:
        print("Error reading file")
    finally:
        return full_doc_text

In [4]:
# Divide the text into chunks of chunk_length 
# [ default is 500] characters
def get_chunks(fulltext:str,chunk_length =500) -> list:
    text = fulltext

    chunks = []
    while len(text) > chunk_length:
        last_period_index = text[:chunk_length].rfind('.')
        if last_period_index == -1:
            last_period_index = chunk_length
        chunks.append(text[:last_period_index])
        text = text[last_period_index+1:]
    chunks.append(text)

    return chunks

In [5]:
filename = FILE_PATH

In [6]:
FILE_PATH.split('/')[-1]

'CHAP04-BIOLOGY-CLASS11.pdf'

In [7]:
full_doc_text = get_pdf_data(filename)

In [8]:
print(f'Full doc text length: {len(full_doc_text)}')

Full doc text length: 31802


In [9]:
Lines =get_chunks(full_doc_text,500)

In [10]:
len(Lines)

73

In [11]:
type(Lines)

list

In [12]:
from openai import AzureOpenAI
import os

In [14]:
load_dotenv()

True

In [16]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [34]:
model: str = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_ID")

In [24]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text, model=model):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [25]:
counter = 0

In [26]:
input_data = []

In [27]:
%%time
for line in Lines:
    d = {}
    d['id'] = str(counter)
    d['line'] = line
    d['embedding'] = generate_embeddings(line)
    d['filename'] = FILE_PATH.split('/')[-1]
    counter = counter + 1
    input_data.append(d)

CPU times: total: 15.6 ms
Wall time: 21.1 s


In [28]:
counter

73

In [31]:
len(input_data[0]['embedding'])

1536

In [33]:
# Output embeddings to docVectors.json file
with open("../output/docVectors_azure.json", "w") as f:
    json.dump(input_data, f)